In [28]:
def is_variable(pat):
    return pat.startswith('?') and all(s.isalpha() for s in pat[1:] )


In [9]:
def pat_match(pattern,saying):
    if is_variable(pattern[0]):
        return True
    else:
        if pattern[0] != saying[0]:
            return False
        else:
            return pat_match(pattern[1:],saying[1:])

In [5]:
help(str.split)

Help on method_descriptor:

split(self, /, sep=None, maxsplit=-1)
    Return a list of the words in the string, using sep as the delimiter string.
    
    sep
      The delimiter according which to split the string.
      None (the default value) means split according to any whitespace,
      and discard empty strings from the result.
    maxsplit
      Maximum number of splits to do.
      -1 (the default value) means no limit.



In [13]:
pat_match('I want holiday'.split(),"I want holiday".split())
#  需限定输入参数

IndexError: list index out of range

In [12]:
pat_match('I want ?x'.split(),"I want holiday".split())

True

In [15]:
def pat_match2(pattern,saying):
    if is_variable(pattern[0]):
        return pattern[0],saying[0]
    else:
        if pattern[0] != saying[0]:return False
        else:
            return pat_match2(pattern[1:],saying[1:])

In [16]:
pattern = 'I want ?x'.split()
saying = 'I want money'.split()

In [17]:
pat_match2(pattern,saying)

('?x', 'money')

In [18]:
pat_match2('I want ?x'.split(),'I want ?x'.split())

('?x', '?x')

In [20]:
pat_match2("?X equals ?Y".split(), "2+2 equals 2+2".split())

('?X', '2+2')

In [2]:
def pat_match3(pattern,saying):
    if not pattern or not saying: return []
    if is_variable(pattern[0]):
        return [(pattern[0],saying[0])] + pat_match3(pattern[1:],saying[1:])
    else:
        if pattern[0] != saying[0]: return []
        else:
            return pat_match3(pattern[1:],saying[1:])

In [28]:
pat_match3("?X equals ?Y".split(), "2+2 equals 2+3".split())

[('?X', '2+2'), ('?Y', '2+3')]

In [30]:
pat_match3("?X greater than ?Y".split(), "3 greater than 2".split())

[('?X', '3'), ('?Y', '2')]

In [37]:
def pat_to_dict(patterns):
    return {k:v for k,v in patterns}

In [38]:
#rule           str   "  ***  ***  **  ?X"
#parsed_rules   dict  { ?x: value ,?Y:VALUE } 
# return list  ?x replaced by value
def subsitite(rule,parsed_rules):
    if not rule:return []
    return [parsed_rules.get(rule[0],rule[0])] + subsitite(rule[1:],parsed_rules)

In [34]:
got_patterns = pat_match3("I want ?X".split(), "I want iPhone".split())

In [35]:
got_patterns

[('?X', 'iPhone')]

In [40]:
subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns))

['What', 'if', 'you', 'mean', 'if', 'you', 'got', 'a', 'iPhone']

In [46]:
' '.join(subsitite("What if you mean if you got a ?X".split(), pat_to_dict(got_patterns)))

'What if you mean if you got a iPhone'

In [44]:
john_pat = pat_match3('?P needs ?X'.split(), "John needs resting".split())
john_pat

[('?P', 'John'), ('?X', 'resting')]

In [47]:
subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat))

['Why', 'does', 'John', 'need', 'resting', '?']

In [48]:
' '.join(subsitite("Why does ?P need ?X ?".split(), pat_to_dict(john_pat)))

'Why does John need resting ?'

In [5]:
defined_patterns = {
    "I need ?X":["Image you will get ?X soon","Why do you need ?X ?"],
    "My ?X told me something":["Talk about more about your ?X", "How do you think about your ?X ?"]
}

In [6]:
def get_response(saying,rules):
    """" please implement the code, to get the response as followings:
    
    >>> get_response('I need iPhone') 
    >>> Image you will get iPhone soon
    >>> get_response("My mother told me something")
    >>> Talk about more about your monther.
    """
    pass

In [30]:
def is_pattern_segment(pattern):
    return pattern.startswith('?*') and all(a.isalpha() for a in pattern[2:] )

In [56]:
is_pattern_segment('?*x你好')

True

In [31]:
is_pattern_segment('?*p')

True

In [20]:
from collections import defaultdict

In [34]:
    def is_match(rest,saying):
        if not rest and not saying:
            return True
        if rest[0]!= '?' and not all(a.isalpha() for a in rest[0]):
            return True
        if rest[0] != saying[0]:
            return False
        return is_match(rest[1:],saying[1:])

In [33]:
    def segment_match(pattern,saying):
        seg_pat,rest = pattern[0],pattern[1:]
        seg_pat = seg_pat.replace('?*','?')

        if not rest : return (seg_pat,saying), len(saying)

        for i,token in enumerate(saying):
            if rest[0] == token and is_match(rest[1:],saying[(i+1):]):
                return (seg_pat,saying[:i]),i

        return (seg_pat,saying),len(saying)

In [57]:
segment_match('?*x你为什么不?*y','小明你为什么不开心')

(('?', '小明你为什么不开心'), 9)

In [26]:
fail = [True,None]
def pat_match_with_seg(pattern,saying):
    if not pattern or not saying: return []

    pat = pattern[0]

    if is_variable(pat):
        return [(pat, saying[0])] + pat_match_with_seg(pattern[1:], saying[1:])
    elif is_pattern_segment(pat):
        match, index = segment_match(pattern, saying)
        if not saying[index:] and pattern[1:]:
            return []
        else:
            return [match] + pat_match_with_seg(pattern[1:], saying[index:])
    elif pat == saying[0]:
        return pat_match_with_seg(pattern[1:], saying[1:])
    else:
        return []

In [108]:
seg_match('?*P is very good'.split(), "My dog and my cat is very good".split())

(('?P', ['My', 'dog', 'and', 'my', 'cat']), 5)

In [109]:
pat_match_with_seg('?*P is very good and ?*X'.split(), "My dog is very good and my cat is very cute".split())

[('?P', ['My', 'dog']), ('?X', ['my', 'cat', 'is', 'very', 'cute'])]

In [14]:
response_pair = {
    'I need ?X':[
        "Why do you need ?X"
    ],
    "I do not like my ?X":[
        "What bad things did ?X do for you ?"
    ]
}

In [25]:
pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())

[('?X', ['an', 'iPhone'])]

In [16]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', ['an', 'iPhone']]

In [15]:
def pat_to_dict(patterns):
    return {k:' '.join(v) if isinstance(v,list) else v for k, v in patterns}

In [28]:
subsitite("Why do you neeed ?X".split(), pat_to_dict(pat_match_with_seg('I need ?*X'.split(), 
                  "I need an iPhone".split())))

['Why', 'do', 'you', 'neeed', 'an iPhone']

In [29]:
subsitite("Hi, how do you do?".split(), pat_to_dict(pat_match_with_seg('?*X hello ?*Y'.split(), 
                  "I am mike, hello ".split())))

['Hi,', 'how', 'do', 'you', 'do?']

In [ ]:
#  题

In [21]:
rule_responses = {
    '?*x hello ?*y': ['How do you do', 'Please state your problem'],
    '?*x I want ?*y': ['what would it mean if you got ?y', 'Why do you want ?y', 'Suppose you got ?y soon'],
    '?*x if ?*y': ['Do you really think its likely that ?y', 'Do you wish that ?y', 'What do you think about ?y', 'Really-- if ?y'],
    '?*x no ?*y': ['why not?', 'You are being a negative', 'Are you saying \'No\' just to be negative?'],
    '?*x I was ?*y': ['Were you really', 'Perhaps I already knew you were ?y', 'Why do you tell me you were ?y now?'],
    '?*x I feel ?*y': ['Do you often feel ?y ?', 'What other feelings do you have?'],
    '?*x你好?*y': ['你好呀', '请告诉我你的问题'],
    '?*x我想?*y': ['你觉得?y有什么意义呢？', '为什么你想?y', '你可以想想你很快就可以?y了'],
    '?*x我想要?*y': ['?x想问你，你觉得?y有什么意义呢?', '为什么你想?y', '?x觉得... 你可以想想你很快就可以有?y了', '你看?x像?y不', '我看你就像?y'],
    '?*x喜欢?*y': ['喜欢?y的哪里？', '?y有什么好的呢？', '你想要?y吗？'],
    '?*x讨厌?*y': ['?y怎么会那么讨厌呢?', '讨厌?y的哪里？', '?y有什么不好呢？', '你不想要?y吗？'],
    '?*xAI?*y': ['你为什么要提AI的事情？', '你为什么觉得AI要解决你的问题？'],
    '?*x机器人?*y': ['你为什么要提机器人的事情？', '你为什么觉得机器人要解决你的问题？'],
    '?*x对不起?*y': ['不用道歉', '你为什么觉得你需要道歉呢?'],
    '?*x我记得?*y': ['你经常会想起这个吗？', '除了?y你还会想起什么吗？', '你为什么和我提起?y'],
    '?*x如果?*y': ['你真的觉得?y会发生吗？', '你希望?y吗?', '真的吗？如果?y的话', '关于?y你怎么想？'],
    '?*x我?*z梦见?*y':['真的吗? --- ?y', '你在醒着的时候，以前想象过?y吗？', '你以前梦见过?y吗'],
    '?*x妈妈?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '她对你影响很大吗？'],
    '?*x爸爸?*y': ['你家里除了?y还有谁?', '嗯嗯，多说一点和你家里有关系的', '他对你影响很大吗？', '每当你想起你爸爸的时候， 你还会想起其他的吗?'],
    '?*x我愿意?*y': ['我可以帮你?y吗？', '你可以解释一下，为什么想?y'],
    '?*x我很难过，因为?*y': ['我听到你这么说， 也很难过', '?y不应该让你这么难过的'],
    '?*x难过?*y': ['我听到你这么说， 也很难过',
                 '不应该让你这么难过的，你觉得你拥有什么，就会不难过?',
                 '你觉得事情变成什么样，你就不难过了?'],
    '?*x就像?*y': ['你觉得?x和?y有什么相似性？', '?x和?y真的有关系吗？', '怎么说？'],
    '?*x和?*y都?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x和?*y一样?*z': ['你觉得?z有什么问题吗?', '?z会对你有什么影响呢?'],
    '?*x我是?*y': ['真的吗？', '?x想告诉你，或许我早就知道你是?y', '你为什么现在才告诉我你是?y'],
    '?*x我是?*y吗': ['如果你是?y会怎么样呢？', '你觉得你是?y吗', '如果你是?y，那一位着什么?'],
    '?*x你是?*y吗':  ['你为什么会对我是不是?y感兴趣?', '那你希望我是?y吗', '你要是喜欢， 我就会是?y'],
    '?*x你是?*y' : ['为什么你觉得我是?y'],
    '?*x因为?*y' : ['?y是真正的原因吗？', '你觉得会有其他原因吗?'],
    '?*x我不能?*y': ['你或许现在就能?*y', '如果你能?*y,会怎样呢？'],
    '?*x我觉得?*y': ['你经常这样感觉吗？', '除了到这个，你还有什么其他的感觉吗？'],
    '?*x我?*y你?*z': ['其实很有可能我们互相?y'],
    '?*x你为什么不?*y': ['你自己为什么不?y', '你觉得我不会?y', '等我心情好了，我就?y'],
    '?*x好的?*y': ['好的', '你是一个很正能量的人'],
    '?*x嗯嗯?*y': ['好的', '你是一个很正能量的人'],
    '?*x不嘛?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x不要?*y': ['为什么不？', '你有一点负能量', '你说 不，是想表达不想的意思吗？'],
    '?*x有些人?*y': ['具体是哪些人呢?'],
    '?*x有的人?*y': ['具体是哪些人呢?'],
    '?*x某些人?*y': ['具体是哪些人呢?'],
    '?*x每个人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x所有人?*y': ['我确定不是人人都是', '你能想到一点特殊情况吗？', '例如谁？', '你看到的其实只是一小部分人'],
    '?*x总是?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x一直?*y': ['你能想到一些其他情况吗?', '例如什么时候?', '你具体是说哪一次？', '真的---总是吗？'],
    '?*x或许?*y': ['你看起来不太确定'],
    '?*x可能?*y': ['你看起来不太确定'],
    '?*x他们是?*y吗？': ['你觉得他们可能不是?y？'],
    '?*x': ['很有趣', '请继续', '我不太确定我很理解你说的, 能稍微详细解释一下吗?']
}

In [22]:
rules = {
    "?*X hello ?*Y": ["Hi, how do you do?"],
    "I was ?*X": ["Were you really ?X ?", "I already knew you were ?X ."],
    "?*X":['Funny!','Continue!']
}

In [34]:
"""
1.  根据我说的话在匹配规则   KEY
2.  根据KEY 随机返回 VALUE
3.  解决中文匹配问题  ,  依据提示 使用jieba分词
4.  先解决英文，再解决中文
"""

'\n1.  根据我说的话在匹配规则   KEY\n2.  根据KEY 随机返回 VALUE\n3.  解决中文匹配问题  ,  依据提示 使用jieba分词\n4.  先解决英文，再解决中文\n'

In [40]:
import random

In [37]:
help(random.choice)

Help on method choice in module random:

choice(seq) method of random.Random instance
    Choose a random element from a non-empty sequence.



In [24]:
# 匹配符合的KEY
def get_matched_key(saying,response_rules):
    matched_key = ''
    for key in response_rules.keys():
        matched_data = pat_match_with_seg(key.split(),saying.split())
        if matched_data:
            matched_key = key
            break
    return  matched_key

def get_variable(saying,key):
    pattern = pat_match_with_seg(key.split(),saying.split())
    return pat_to_dict(pattern)
def get_response_data(pattern:str,response_dict:dict):
    return  subsitite(pattern.split(),response_dict)

In [19]:
def get_response(saying:str,response_rules:dict):
    if not saying or not response_rules:return ''
    
    #  匹配规则中的key
    matched_key = get_matched_key(saying,response_rules)
    #  提取KEY variable
    
    response_dict = pat_to_dict(pat_match_with_seg(matched_key.split(),saying.split()))
    #随机返回 应答的规则
    random_pattern =  random.choice(response_rules[matched_key])
    
    response_data =   subsitite(random_pattern.split(),response_dict)
    #response_data = []
    return ' '.join(response_data)
    

In [41]:
# test  en  response
get_response("I am XiaoMing, hello",rules)


'Hi, how do you do?'

In [43]:
get_response("I was XiaoMing",rules)

TypeError: sequence item 3: expected str instance, list found

In [44]:
get_response("I was XiaoMing",rules)

TypeError: sequence item 5: expected str instance, list found

In [45]:
get_response("super star",rules)

'Funny!'

In [47]:
get_response("super star",rules)

'Continue!'

In [2]:
#   next deal with CN
#about   jieba
import jieba

In [8]:
words = jieba.cut('?*x一直?*y')

In [9]:
for x in words:
    print(x)

?
*
x
一直
?
*
y


In [10]:
jieba.cut?

In [13]:
words = jieba.cut("?*x I was ?*y")

In [14]:
for x in words:
    print(x)

?
*
x
 
I
 
was
 
?
*
y


In [15]:
def cut(string:str):
    lstRet = jieba.cut(string)
    return list(lstRet)

In [17]:
cut('?*d你好呀，kk')

['?', '*', 'd', '你好', '呀', '，', 'kk']

In [58]:
cut('你好呀，')

['你好', '呀', '，']

In [ ]:
def is_variable_CN(words:list):
    return True

In [52]:
from  collections import Counter

In [60]:
string = '?*x abc 你好 ?*y'

In [51]:
if '?*x' in string:
    print(True)

True


In [55]:
Counter?